# **PROBLEM 1: Home Court Advantage** 🏡
Does playing on your own home field provide any sort of noticeable and quantifiable advantage?

Season game stats are available from the 2014 season onwards. In that time the league grew from X teams to 25 teams.

Approach:


In [47]:
# IMPORTS
import os
import pandas as pd
import seaborn as sns
import re

In [7]:
# Sample Data
data_path = ".\\DATA\\"
df = pd.read_csv(data_path + '2014SEASON_GAME_STATS.csv')
df.head(10)

,Team,Date ▼,Score,Completion %,Hold %,OLC %,Break %,DLC %,Turnovers,Blocks
0,Spiders,"July 27, 2014",SJ 28 - 18 TOR,91% (234/257),85% (17/20),61% (17/28),37% (11/30),48% (11/23),23,10
1,Rush,"July 26, 2014",NY 16 - 31 TOR,92% (270/292),89% (16/18),64% (16/25),45% (15/33),50% (15/30),22,19
2,Empire,"July 26, 2014",NY 16 - 31 TOR,85% (213/250),45% (15/33),32% (15/47),6% (1/18),13% (1/8),37,12
3,Spiders,"July 26, 2014",MAD 20 - 23 SJ,93% (272/291),73% (16/22),57% (16/28),28% (7/25),47% (7/15),19,10
4,Radicals,"July 26, 2014",MAD 20 - 23 SJ,93% (265/286),60% (15/25),45% (15/33),23% (5/22),42% (5/12),21,16
5,Radicals,"July 19, 2014",IND 16 - 25 MAD,94% (293/312),78% (14/18),56% (14/25),41% (11/27),48% (11/23),19,22
6,AlleyCats,"July 19, 2014",IND 16 - 25 MAD,91% (288/316),48% (13/27),35% (13/37),17% (3/18),30% (3/10),28,6
7,Breeze,"July 19, 2014",DC 12 - 37 TOR,86% (243/283),28% (11/39),23% (11/48),7% (1/14),25% (1/4),40,5
8,Rush,"July 19, 2014",DC 12 - 37 TOR,95% (253/266),85% (11/13),73% (11/15),67% (26/39),68% (26/38),13,13
9,Empire,"July 19, 2014",CHI 17 - 22 NY,94% (289/308),68% (13/19),50% (13/26),38% (9/24),53% (9/17),19,16


**Data Cleaning:**
- Create a dictionary with team name (e.g. Empire) and shorthand (e.g. NY) ✅
- Parse Score into home and away score. Keep appropriate score ✅
- Create a function to parse completions and attempts for multiple attributes ✅
- Parse completions and comp_attempts ✅
- Parse holds and hold_chances ✅
- OLC stands for O-Line Conversion, parse offensive_conversions and offensive_points ✅
- Parse breaks and break_chances ✅
- DLC stands for D-Line Conversions, parse deffensive_conversions and deffensive_points ✅
- Assign home team flag
- change column names

In [60]:
# Drop Date Column
df.drop(labels='Date ▼', axis=1, inplace=True)

In [39]:
# Create dictionary with team name and shorthand
team_shorthand = {
    'SJ': 'Spiders',
    'TOR': 'Rush',
    'NY': 'Empire',
    'MAD': 'Radicals',
    'IND': 'AlleyCats',
    'DC': 'Breeze',
    'CHI': 'Union',
    'SF': 'FlameThrowers',
    'MIN': 'Wind Chill',
    'SEA': 'Cascades',
    'VAN': 'Riptide',
    'PHI': 'Phoenix',
    'ROC': 'Dragons',
    'MTL': 'Royal',
    'DET': 'Mechanix',
    'CIN': 'Revolution',
    'SLC': 'Lions'
}

In [132]:
# Parse the double score text
def parse_double_score(score_string: str) -> dict:
    """
    Through online validation, it appears that the score strings show scores as <away> <away_score> - <home_score> <home>
    We can parse these and retun all using split. String example: 'IND 16 - 25 MAD', Where Madison (MAD) is the home team
    that scored 25 points and Indianapolis (IND) is the away team that scored 16 points.

    INPUT
    -----
    score_string: A string denoting the score of a game as well as the home and away teams

    OUTPUT:
    -----
    return_dict: A dictionary with the parsed score and team information
    """

    elements = score_string.split(' ')
    if len(elements) < 5:
        print(f"Unable to parse string '{score_string}'")
        return None
    
    return_dict = {
        'home_team': elements[4],
        'home_score': int(elements[3]),
        'away_team': elements[0],
        'away_score': int(elements[1])
    }

    return return_dict


# Parse individual attributes
def parse_attribute_attempts(attribute_str: str) -> [int, int]:

    """
    Return the number of attempts for an attribute and the number of completions
    
    INPUT:
    -----
    attribute_str: Attribute completion string (e.g. '91% (234/257)')

    OUTPUT:
    -----
    completions: integr
    attempts: integer (>= completions)
    """
    
    completions = re.search('\(([^)]+)', attribute_str).group(1).split('/')[0]
    attempts = re.search('\(([^)]+)', attribute_str).group(1).split('/')[1]
    return int(completions), int(attempts)


# Parse completions and attempts
def process_df_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process a DataFrame to pull out info from completion, hold, break, and red zone columns.

    INPUT:
    -----
    df: pandas dataframe with 'Completion %', 'Hold %', 'Break %', and/or 'RZC %' columns

    OUTPUT:
    df: pandas dataframe with parsed columns into integers
    """
    if 'Completion %' in df.columns:
        df['completions'] = df['Completion %'].apply(lambda x: parse_attribute_attempts(x)[0])
        df['total_passes'] = df['Completion %'].apply(lambda x: parse_attribute_attempts(x)[1])
    else:
        df['completions'] = df['total_passes'] = pd.NA

    if 'Hold %' in df.columns:
        df['o_line_scores'] = df['Hold %'].apply(lambda x: parse_attribute_attempts(x)[0])
        df['o_line_points'] = df['Hold %'].apply(lambda x: parse_attribute_attempts(x)[1])
        df['o_line_posessions'] = df['OLC %'].apply(lambda x: parse_attribute_attempts(x)[1])
    else:
        df['o_line_scores'] = df['o_line_points'] = df['o_line_posessions'] = pd.NA

    if 'Break %' in df.columns:
        df['d_line_scores'] = df['Break %'].apply(lambda x: parse_attribute_attempts(x)[0])
        df['d_line_points'] = df['Break %'].apply(lambda x: parse_attribute_attempts(x)[1])
        df['d_line_posessions'] = df['DLC %'].apply(lambda x: parse_attribute_attempts(x)[1])
    else:
        df['d_line_scores'] = df['d_line_points'] = df['d_line_posessions'] = pd.NA

    if 'RZC %' in df.columns:
        df['rz_scores'] = df['RZC %'].apply(lambda x: parse_attribute_attempts(x)[0])
        df['rz_posessions'] = df['RZC %'].apply(lambda x: parse_attribute_attempts(x)[1])
    else:
        df['rz_scores'] = df['rz_posessions'] = pd.NA

    return df

In [73]:
df = process_df_attributes(df)
df.sample(5)

,Team,Score,Completion %,Hold %,OLC %,Break %,DLC %,Turnovers,Blocks,home_score,...,completions,total_passes,o_line_scores,o_line_points,o_line_posessions,d_line_scores,d_line_points,d_line_posessions,rz_scores,rz_posessions
23,Phoenix,DC 25 - 20 PHI,93% (325/351),62% (16/26),52% (16/31),18% (4/22),25% (4/16),26,13,20,...,325,351,16,26,31,4,22,16,<NA>,<NA>
4,Radicals,MAD 20 - 23 SJ,93% (265/286),60% (15/25),45% (15/33),23% (5/22),42% (5/12),21,16,23,...,265,286,15,25,33,5,22,12,<NA>,<NA>
234,AlleyCats,MAD 26 - 21 IND,88% (210/238),68% (19/28),48% (19/40),9% (2/23),17% (2/12),28,15,21,...,210,238,19,28,40,2,23,12,<NA>,<NA>
15,Cascades,VAN 21 - 18 SEA,88% (211/239),52% (12/23),38% (12/32),30% (6/20),40% (6/15),28,13,18,...,211,239,12,23,32,6,20,15,<NA>,<NA>
87,Radicals,MIN 15 - 23 MAD,92% (245/266),71% (12/17),52% (12/23),44% (11/25),44% (11/25),21,18,23,...,245,266,12,17,23,11,25,25,<NA>,<NA>


In [74]:
drop_labels = ['Score', 'Completion %', 'Hold %', 'OLC %', 'Break %', 'DLC %']
df.drop(labels=drop_labels, axis=1, inplace=True)
df.sample(5)

,Team,Turnovers,Blocks,home_score,away_score,home_team,away_team,completions,total_passes,o_line_scores,o_line_points,o_line_posessions,d_line_scores,d_line_points,d_line_posessions,rz_scores,rz_posessions
205,FlameThrowers,29,14,13,17,SEA,SF,241,270,7,15,25,10,19,23,<NA>,<NA>
93,Dragons,40,8,38,18,TOR,ROC,270,310,12,40,49,6,20,10,<NA>,<NA>
8,Rush,13,13,37,12,TOR,DC,253,266,11,13,15,26,39,38,<NA>,<NA>
224,AlleyCats,15,15,25,19,IND,CHI,243,258,16,21,25,9,27,18,<NA>,<NA>
53,Cascades,24,8,14,22,SEA,SJ,254,278,12,24,31,2,16,9,<NA>,<NA>


In [143]:
def check_hometeam(teamname, shorthand):
    return teamname == team_shorthand[shorthand]

df['is_home_team'] = pd.NA
f_count = 0
t_count = 0
for i in range(len(df)):
    full_name = df.at[i, 'Team']
    shorthand = df.at[i, 'home_team']

    df.at[i, 'is_home_team'] = check_hometeam(full_name, shorthand)

df['is_home_team'].value_counts()  


False    125
True     123
Name: is_home_team, dtype: int64

This is interesting. Based on the way the data was parsed, there should be an equal number of False/True in this column. Upon reviewing the data, it's noted that as expected, in most cases each game is reported twice, each row from the perspective of one of the two teams. The following games appear to only have been reported once:

- May 18, 2014: SLC Lions @ SEA Cascade. This game is reported with the stats for the Cascades, but not the Lions
- July 27, 2014: SJ Spiders @ TOR Rush. This is the championship game and the stats are only posted from the Spiders perspective.

The fact that the stats are missing from the perspective of two teams is likely what is causing the mismatch of False/True values. The May 18th game is reported from the perspective of the Home team, so the 'is_home_team' column gets +1 True value from there. Additionally, for the July 27th game, it is only reported from the perspective of the Away team, so the 'is_home_team' column gets +1 False value from there. 

TODO: Investigate above mismatch... what the actual f$#^?

In [142]:
for i in team_shorthand.keys():
    h_count = len(df[df['home_team'] == i])
    a_count = len(df[df['away_team'] == i])

    print(f"{i}: {h_count - a_count}")

SJ: 3
TOR: 5
NY: 0
MAD: 0
IND: -2
DC: -2
CHI: -2
SF: -2
MIN: 0
SEA: 0
VAN: -1
PHI: 0
ROC: 0
MTL: 0
DET: 0
CIN: 0
SLC: 1
